# Step by step reconstruction of the model

In [2]:
import numpy as np
import tensorflow as tf
import os
import cv2

In [6]:
def load_data(img_dir, ending):
    return np.array([cv2.imread(os.path.join(img_dir, img)) for img in os.listdir(img_dir) if img.endswith(ending)])


In [129]:
X = load_data("3Shapes2_large/", "im1.png")
Y = load_data("3Shapes2_large/","im2.png")

In [130]:
X.shape

(8015, 128, 128, 3)

In [194]:
X = X[:10]
Y = Y[:10]

In [199]:
def create_graph(img1, img2, batch_size):
    img1_64 = tf.nn.max_pool(img1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    img2_64 = tf.nn.max_pool(img2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    #Motion encoder

    #First convolution: 5x5x96
    weights = tf.Variable(tf.random_normal([5,5,3,96]))
    bias = tf.Variable(tf.zeros([96,]))
    logits = tf.nn.conv2d(img1_64, filter=weights, strides=[1,1,1,1], padding='SAME')
    logits = tf.add(logits, bias)
    logits = tf.nn.relu(logits)
    
    #Second convolution: 5x5x96
    weights = tf.Variable(tf.random_normal([5,5,96,96]))
    bias = tf.Variable(tf.zeros([96,]))
    logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='SAME')
    logits = tf.add(logits, bias)
    logits = tf.nn.max_pool(logits, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    logits = tf.nn.relu(logits)
    
    #Third convolution: 5x5x128
    weights = tf.Variable(tf.random_normal([5,5,96,128])) # 128
    bias = tf.Variable(tf.zeros([128,]))
    logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='SAME')
    logits = tf.add(logits, bias)
    logits = tf.nn.max_pool(logits, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    logits = tf.nn.relu(logits)
    
    #Forth convolution: 5x5x128
    weights = tf.Variable(tf.random_normal([5,5,128,128]))
    bias = tf.Variable(tf.zeros([128,]))
    logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='VALID')
    logits = tf.add(logits, bias)
    logits = tf.nn.max_pool(logits, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    logits = tf.nn.relu(logits)

    #Fifth convolution: 5x5x256
    weights = tf.Variable(tf.random_normal([5,5,128,256])) #256
    bias = tf.Variable(tf.zeros([256,]))
    logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='VALID')
    logits = tf.add(logits, bias)
    logits = tf.nn.max_pool(logits, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    logits = tf.nn.relu(logits)

    #Sixth convolution: 5x5x256
    weights = tf.Variable(tf.random_normal([5,5,256,3])) # TODO change 3 for 256
    bias = tf.Variable(tf.zeros([3,]))
    logits = tf.nn.conv2d(logits, filter=weights, strides=[1,1,1,1], padding='SAME')
    logits = tf.add(logits, bias)
    logits = tf.nn.relu(logits)
    tf.Print(logits,[logits])
    
    logits = tf.image.resize_images(logits, [64,128])
    
    #flatten
    logits_flatten = tf.reshape(logits, shape=[logits.shape.as_list()[0],-1])
    
    # mean and std
    mean, std_log = tf.split(logits_flatten, 2, axis=1)
    epsilon = tf.random_normal(mean.shape.as_list(),0,1, dtype=tf.float32)
    kernel = mean + tf.multiply(tf.exp(std_log),epsilon)
    
    kernel = tf.reshape(kernel, shape=[batch_size,64,64,3])
    
    return img1_64, img2_64, kernel


In [200]:
def train(prediction, y):
    l2_loss = tf.reduce_mean(tf.square(prediction - y))
    optimizer = tf.train.AdamOptimizer(0.1)
    train = optimizer.minimize(l2_loss)
    
    return l2_loss, train

In [201]:
# run the model
def run(X, Y, n_epochs = 10, batch_size = 30):
    img1 = tf.placeholder(shape=(batch_size,128,128,3), dtype=tf.float32, name="s1s")
    img2 = tf.placeholder(shape=(batch_size,128,128,3), dtype=tf.float32)
    
    img1_64, img2_64, output = create_graph(img1, img2, batch_size)
    loss, training = train(output, img2_64)
    
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(n_epochs):
            
            print('Epoch %i/%i' % (epoch+1, n_epochs))
            cumulative_loss = 0.0
            
            for batch_num in range(int(X.shape[0]/batch_size)-1):
                # get x and y
                from_index = batch_size*batch_num
                to_index = batch_size*batch_num + batch_size
                x = X[from_index:to_index]
                y = Y[from_index:to_index]
                
                # run train and loss
                _, batch_loss = sess.run([training,loss], feed_dict={img1:x, img2:y})
                print("\t\tbatch_loss:", batch_loss)
                cumulative_loss += batch_loss
            
            print("\tEpoch's loss:", cumulative_loss)
                

# TODO
1. Solve the nan problem
2. Integrate different sizes

In [202]:
run(X, Y, 10, 2)

Epoch 1/10
		batch_loss: inf
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 2/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 3/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 4/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 5/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 6/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 7/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 8/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 9/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
	Epoch's loss: nan
Epoch 10/10
		batch_loss: nan
		batch_loss: nan
		batch_loss: nan
		batch_loss: na